In [0]:
%pip install --quiet mlflow==2.22.0
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
!pip install hyperopt

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from mlflow.models import Model, infer_signature, ModelSignature

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [0]:
dbutils.widgets.text("catalog", "mlops")
dbutils.widgets.text("schema", "mlops_zoomcamp_prj")
dbutils.widgets.text("tbl_name", "soft_quality_features")


catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
tbl_name = dbutils.widgets.get("tbl_name")

In [0]:
def transform_df_to_matrix(df, dv):    
    numerical_columns = list(df.columns)
    numerical_columns.remove('defects')
    dct = df[numerical_columns].to_dict(orient='records')    
    X = dv.fit_transform(dct)
    Y = df.defects.values

    return X,Y
    

def train(model_params, X_train, y_train):

    model = RandomForestClassifier(n_estimators=model_params['n_estimators'] , max_depth=model_params['max_depth'] ,random_state=1)
    model.fit(X_train, y_train)
    
    return model

# df, numerical_columns = load_df(data_file_path)
# dv, model = train(df,model_params, numerical_columns)


In [0]:


import mlflow
from mlflow import MlflowClient

mlflow.set_registry_uri("databricks-uc")

In [0]:
# read dataset from delta-table
df_train = spark.sql(f"""
               select * except (split) 
               from
               {catalog}.{schema}.{tbl_name}
               where split = 'train'
               """).toPandas()
df_test = spark.sql(f"""
               select * except (split) 
               from
               {catalog}.{schema}.{tbl_name}
               where split = 'test'
               """).toPandas()

# split dataset into Train(80%) and Test(20%)   
# df_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

dv = DictVectorizer(sparse=False)

X_train, y_train = transform_df_to_matrix(df_train, dv)
X_test, y_test = transform_df_to_matrix(df_test, dv)



In [0]:
current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
print(current_user)

dmitriy.shametko@gmail.com


In [0]:
# Set UC Model Registry as default
mlflow.set_registry_uri("databricks-uc")
xp_name = "software-defects"
xp_path = f"/Users/{current_user}/mlops_prj"
try:
  from databricks.sdk import WorkspaceClient
  w = WorkspaceClient()
  r = w.workspace.mkdirs(path=xp_path)
except Exception as e:
  print(f"ERROR: couldn't create a folder for the experiment under {xp_path} - please create the folder manually or  skip this init (used for job only: {e})")
  raise e

client = MlflowClient()

In [0]:

mlflow.set_experiment(f'{xp_path}/{xp_name}')

2025/07/12 10:25:05 INFO mlflow.tracking.fluent: Experiment with name '/Users/dmitriy.shametko@gmail.com/mlops_prj/software-defects' does not exist. Creating a new experiment.


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/207411282372909', creation_time=1752315905712, experiment_id='207411282372909', last_update_time=1752315905712, lifecycle_stage='active', name='/Users/dmitriy.shametko@gmail.com/mlops_prj/software-defects', tags={'mlflow.experiment.sourceName': '/Users/dmitriy.shametko@gmail.com/mlops_prj/software-defects',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'dmitriy.shametko@gmail.com',
 'mlflow.ownerId': '3626367561960979'}>

In [0]:
# with mlflow.start_run():

#     # X, y = load_iris(return_X_y=True)

#     # params = {"C": 0.1, "random_state": 42}
#     model_params = {
#         'n_estimators':140,
#         'max_depth':30,
#         'threeshold': 0.35
#     }

#     mlflow.log_params(model_params)
    
#     model = train(model_params, X_train, y_train)

#     # lr = LogisticRegression(**params).fit(X, y)
#     y_pred = model.predict_proba(X_test)[:,1]

#     # y_pred = lr.predict(X)

#     mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred > model_params['threeshold']))

#     # print(y_pred)


#     mlflow.sklearn.log_model(model, artifact_path="models")
#     print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

In [0]:
def objective(params):
    
    with mlflow.start_run():
     
        mlflow.log_params(params)
        
        model = train(params, X_train, y_train)

        signature = infer_signature(X_train, y_train)
        mlflow.sklearn.log_model(model, "sklearn_model",  signature=signature)
        
        y_pred = model.predict_proba(X_test)[:,1]    

        accuracy =  accuracy_score(y_test, y_pred > params['threeshold'])    
        loss = -accuracy

        mlflow.log_metric("accuracy", accuracy)     

    return {'loss': loss, 'status': STATUS_OK}

In [0]:
search_space = {
        'n_estimators': hp.choice('n_estimators', range(10, 201, 10)),
        'max_depth':hp.choice('max_depth', [1, 2, 3, 4, 5, 6, 7,8 ,9,10, 15, 20, 25, 30, None]),
        'threeshold': hp.choice('threshold', [round(x * 0.05, 2) for x in range(21)])
    }


trials = Trials()

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)


  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

  1%|          | 1/100 [00:07<11:43,  7.11s/trial, best loss: -0.8641304347826086]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

  2%|▏         | 2/100 [00:12<09:53,  6.06s/trial, best loss: -0.8641304347826086]

...

 98%|█████████▊| 98/100 [08:14<00:11,  5.83s/trial, best loss: -0.8967391304347826]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

 99%|█████████▉| 99/100 [08:19<00:05,  5.50s/trial, best loss: -0.8967391304347826]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 100/100 [08:24<00:00,  5.04s/trial, best loss: -0.8967391304347826]


In [0]:

trials.best_trial

{'state': 2,
 'tid': 31,
 'spec': None,
 'result': {'loss': -0.8967391304347826, 'status': 'ok'},
 'misc': {'tid': 31,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'max_depth': [31], 'n_estimators': [31], 'threshold': [31]},
  'vals': {'max_depth': [5], 'n_estimators': [3], 'threshold': [10]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2025, 7, 12, 10, 27, 57, 105000),
 'refresh_time': datetime.datetime(2025, 7, 12, 10, 28, 1, 738000)}